In [ ]:
!pip install transformers
!pip install pydub
!pip install datasets
!pip install librosa
!pip install tqdm

In [ ]:
from transformers import pipeline
import pandas as pd
from datasets import load_metric
from datasets import load_dataset
from transformers import Wav2Vec2CTCTokenizer
from glob import glob
import wave
import os
import torch
import re
import torchaudio
import librosa
from tqdm import tqdm
import requests
import json
from scipy.io.wavfile import read, write
import io

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
paths = glob('/content/drive/MyDrive/new_dataset/ready2train2/*.mp3')
for path in paths:
  new_path = path[:-4]
  # print(path,f'{new_path}.wav')
  os.rename(path,f'{new_path}.wav')

In [ ]:
ktp = pipeline(model="Kittipong/wav2vec2-th-vocal-domain")
air = pipeline(model="airesearch/wav2vec2-large-xlsr-53-th")

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/new_dataset/test.tsv',sep="\t")
products_list = df.values.tolist()
all_predict1 = []

for i in tqdm(products_list, desc='products_list') :
  
  name, lyric= i[0], i[1]
  if name[:4]!="Kiss":
    song_path = f'/content/drive/MyDrive/new_dataset/ready2train/{name}'
    data = open(f'{song_path}', 'rb').read()
  
    m = ktp(data)
    n = air(data)
    print(m,n)
    all_predict1.extend([{
        "song_name" : name,
        "actual" : lyric,
        "predict_ktp" : m,
        "predict_aire" : n
    }])
  


In [ ]:
pd.DataFrame(all_predict).to_csv("/content/data.csv",sep=",")

In [ ]:
df = pd.DataFrame(all_predict1)
df.to_csv("/content/dataframe.csv")

In [ ]:
products_list = df.values.tolist()

In [ ]:
!wget https://github.com/PyThaiNLP/wisesight-sentiment/archive/master.zip; unzip master.zip
!mv wisesight-sentiment-master/kaggle-competition/* .
!pip -q install tensorflow_text
!pip -q install pythainlp==3.0.0
!pip -q install emoji
!pip -q install python-crfsuite

In [ ]:
from pythainlp import word_tokenize

In [ ]:
def process_text(text):
    res = text.lower().strip() 
    res = [word for word in word_tokenize(res) if word and not re.search(pattern=r"\s+", string=word)]

    return res

In [ ]:
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 3.6 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149870 sha256=ab0089f8f732e678b96eebb7ed4e25804b1c1a7ce2a5cc3bc498b06b0c6233ed
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [ ]:
!pip install evaluate

In [ ]:
from evaluate import load

In [ ]:
wer_metric = load("wer")

In [ ]:
cer_metric = load("cer")

In [ ]:
df = pd.read_csv('/content/dataframe.csv')
products_list = df.values.tolist()

In [ ]:
all_data=[]
n=0
for i in products_list:
  name = i[0]
  temp = i[2]
  pred_vis = i[4]
  pred_ktp = i[3]
  pred_vis = pred_vis[10:-2]
  pred_ktp = pred_ktp[10:-2]
  temp = process_text(temp.replace(" ",""))
  pred_vis1 = process_text(pred_vis.replace(" ",""))
  pred_ktp1 = process_text(pred_ktp.replace(" ",""))
  temp = " ".join([f"{x}" for x in temp])
  pred_vis2 = " ".join([f"{spell(x)[0]}" for x in pred_vis1])
  pred_ktp2 = " ".join([f"{spell(x)[0]}" for x in pred_ktp1])
  pred_vis1 = " ".join([f"{x}" for x in pred_vis1])
  pred_ktp1 = " ".join([f"{x}" for x in pred_ktp1])
  all_data.extend([{
      'name':name,
      'temp':temp,
      'pred_vis':pred_vis1,
      'pred_ktp':pred_ktp1,
      'wer_vis': wer_metric.compute(predictions=[f'{pred_vis1}'],references=[f'{temp}']),
      'wer_ktp': wer_metric.compute(predictions=[f'{pred_ktp1}'],references=[f'{temp}']),
      'cer_vis': cer_metric.compute(predictions=[f'{pred_vis1}'],references=[f'{temp}']),
      'cer_ktp': cer_metric.compute(predictions=[f'{pred_ktp1}'],references=[f'{temp}']),
      'spc_pred_vis':pred_vis2,
      'spc_pred_ktp':pred_ktp2,
      'spc_wer_vis': wer_metric.compute(predictions=[f'{pred_vis2}'],references=[f'{temp}']),
      'spc_wer_ktp': wer_metric.compute(predictions=[f'{pred_ktp2}'],references=[f'{temp}']),
      'spc_cer_vis': cer_metric.compute(predictions=[f'{pred_vis2}'],references=[f'{temp}']),
      'spc_cer_ktp': cer_metric.compute(predictions=[f'{pred_ktp2}'],references=[f'{temp}'])
  }])
  print(n)
  n+=1
  

In [ ]:
df = pd.DataFrame(all_data)
df

,name,temp,pred_vis,pred_ktp,wer_vis,wer_ktp,cer_vis,cer_ktp,spc_pred_vis,spc_pred_ktp,spc_wer_vis,spc_wer_ktp,spc_cer_vis,spc_cer_ktp
0,0,แม้ว่า ฉัน เอง จะ เป็น คนเดียว,แล้ว ฉัน จะ เป็น ด็กนิ้ง ของ เด็ด,แม้ว่า ฉัน จะ เป็น เพียง คนเดียว,0.833333,0.333333,0.700000,0.333333,แล้ว ฉัน จะ เป็น ด็กนิ้ง ของ เด็ด,แม้ว่า ฉัน จะ เป็น เพียง เดียว,0.833333,0.500000,0.700000,0.333333
1,1,และ ฉัน ก็ คิด อยาก มีสิทธิ์ มากกว่า นั้น,ฉัน เกรด กรี บ ยังมี เซ ษมา กก ว่า นั้น,แต่ ฉัน คิด คิด ยังมี เสิทธิ์ มากกว่า นั้น,1.125000,0.500000,0.585366,0.268293,ฉัน เกรด กรี พบ ยัง เซ ษมา กก ว่า นั้น,แต่ ฉัน คิด คิด ยัง สิทธิ์ หากว่า นั้น,1.125000,0.625000,0.609756,0.292683
2,2,ความรัก ทำให้ หลงรัก,ความรัก ทำให้ หลงรัก,ความรัก ทำให้ หลง ลัก,0.000000,0.666667,0.000000,0.100000,ความรู้ ทำให้ ลงรัก,ความรู้ ทำให้ หลง ลัก,0.666667,1.000000,0.150000,0.200000
3,3,เพียง เธอ นั้น ใช้ แค่ คำ หนึ่ง คำ รั้ง ไม่ ให...,เพียง ธา นั่ง ใช้ เคด คำ หนึ ทาง รัง เม ์ ให้ ฉัน,เพียง เธอ นั้น ใช้ แค่ คำ หนึ่ง ทำ ร มี ให้ ฉั...,0.692308,0.230769,0.333333,0.111111,เพียง ธา นั่ง ใช้ เคด คำ หนึ ทาง รัง เม ใน ให้...,เพียง เธอ นั้น ใช้ แค่ คำ หนึ่ง ทำ รถ มี ให้ ฉ...,0.692308,0.230769,0.351852,0.111111
4,4,น้ำตา ที่ ยัง ริน ไหล,ล้าง เส ีง ริน ไหล่,นาน แต่ ที่ ยัง ดิ้น ไหล,0.800000,0.600000,0.476190,0.333333,ล้าง เส ลง ริน ไหล่,นาน แต่ ที่ ยัง ดิ้น ไหล,0.800000,0.600000,0.476190,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,421,ยิ่ง ทำให้ ใจ ฉัน มัน ยิ่ง ทรมาน,ิง ทำ หี ใจ ชื่ง ยิง ชอ ล้ำ หา,ยิ่ง ทำให้ ใจ ฉัน ยิ่ง ทร มา,1.142857,0.428571,0.625000,0.187500,ลง ทำ หี ใจ ชื่อ ยิง ชอ ล้ำ หา,ยิ่ง ทำให้ ใจ ฉัน ยิ่ง ทร มา,1.142857,0.428571,0.656250,0.187500
422,422,แต่ ก็ เหมือนว่า แค่ แกล้งทำ,เด กำ เนิน ว่า แค่ ไกล เถิ,แต่ ก็ หมืน ว่า แค่ แก ลง ทำ,1.200000,1.000000,0.500000,0.214286,เด กำ เนิน ว่า แค่ ไกล เถิด,แต่ ก็ หมืน ว่า แค่ แก ลง ทำ,1.200000,1.000000,0.535714,0.214286
423,423,อากาศ ชื้น นะ คืนนี้ ระวัง หนาว นะ แบบนี้,อต ยาก ชิ้น นึถืน นี้ โรง วัล ดาว น้ แบบนี้,อาจ การ ชิ้น เมื่อคืนนี้ รางวัล หนาว แม่ แบบนี้,1.125000,0.750000,0.390244,0.365854,อา ยาก ชิ้น นืน นี้ โรง วัล ดาว นี้ แค่นี้,อาจ การ ชิ้น เมื่อคืนนี้ รางวัล หนาว แม่ แค่นี้,1.250000,0.875000,0.439024,0.414634
424,424,ว่า ฉันนั้น เป็น อย่างไร,ว่า ฉัน น เป็น อย่างไร,ว่า ฉันนั้น เป็น อย่างไร,0.500000,0.000000,0.125000,0.000000,ว่า ฉัน ใน เป็น อย่างไร,ว่า ฉันนั้น เป็น อย่างไร,0.500000,0.000000,0.125000,0.000000


In [ ]:
df.to_csv("/content/dataframe1.csv")

In [ ]:
from pythainlp import spell
spell("รั๊ว")[0]

'รั้ว'

In [ ]:
df= pd.read_csv("/content/data.csv",sep=",")
products_list = df.values.tolist()
n=1
all_ktp,all_vistec=0,0
all_data = []
for i in products_list:
    name,predicted_vistec, predicted_ktp, temp= i[1],i[3], i[4], i[2]
    temp = process_text(temp)
    temp = " ".join([f"{x}" for x in temp])
    wer_ktp = wer_metric.compute(predictions=[f'{temp}'],references=[f'{predicted_ktp[10:-2]}'])
    wer_vistec = wer_metric.compute(predictions=[f'{temp}'],references=[f'{predicted_vistec[10:-2]}'])
    all_ktp+=wer_ktp
    all_vistec+=wer_vistec
    all_data.extend([{
        "name_song" : name,
        "template" : temp,
        "predict_vistec" : predicted_vistec[10:-2],
        "predict_ktp" : predicted_ktp[10:-2],
        "wer_vistec" : wer_vistec,
        "wer_ktp" : wer_ktp
    }])
    # print(f'{wer_ktp} {wer_vistec}')
    n+=1
    # print(f'{temp}===={predicted_vistec[10:-2]}==={predicted_ktp[10:-2]}')
# print(f'{all_ktp/n} {all_vistec/n}') 

In [ ]:
all_data = pd.DataFrame(all_data)
all_data

,name,temp,pred_vis,pred_ktp,wer_vis,wer_ktp,cer_vis,cer_ktp,spc_pred_vis,spc_pred_ktp,spc_wer_vis,spc_wer_ktp,spc_cer_vis,spc_cer_ktp
0,0,แม้ว่า ฉัน เอง จะ เป็น คนเดียว,มา ฉัน คิด ถั้ง นัก มือ น เด ิ,มอง ฉัน คี่ วัน ด เหมือน นเดิวน,1.333333,1.000000,0.766667,0.700000,มา ฉัน คิด ทั้ง นัก มือ ใน เด สิ,มอง ฉัน คี่ วัน ดี เหมือน เดิน,1.333333,1.000000,0.766667,0.766667
1,1,และ ฉัน ก็ คิด อยาก มีสิทธิ์ มากกว่า นั้น,ฉัน เก รก รีต อย่าง มี เส็ต หมาก กว่า นั้น,แต่ ฉัน คิด คิด ยัง มีสิทธิ์ มอ กก ว่า นั้น,1.125000,0.750000,0.512195,0.268293,ฉัน เก รก รีต อย่าง มี เน็ต หมาก กว่า นั้น,แต่ ฉัน คิด คิด ยัง สิทธิ์ มอ กก ว่า นั้น,1.125000,0.875000,0.512195,0.317073
2,2,ความรัก ทำให้ หลงรัก,ความรัก ทำให้ โรง รัก,ควม รัก ทำให้ หลงรัก,0.666667,0.666667,0.150000,0.100000,ความรู้ ทำให้ โรง รัก,ควม รัก ทำให้ ลงรัก,1.000000,1.000000,0.250000,0.150000
3,3,เพียง เธอ นั้น ใช้ แค่ คำ หนึ่ง คำ รั้ง ไม่ ให...,หืมธอ น้อง ใช้ คือ ค อำ หนี ทอง ดอง มอ ให้ ฉัน,เพื่อน ธอ น้อง ชา ก็ คง มัน คง ร้อง มือ น ให้ ...,0.846154,0.846154,0.537037,0.500000,หมอ น้อง ใช้ คือ คน อำ หนี ทอง ดอง มอ ให้ ฉัน,เพื่อน ธอ น้อง ชา ก็ คง มัน คง ร้อง มือ ใน ให้...,0.846154,0.846154,0.574074,0.518519
4,4,น้ำตา ที่ ยัง ริน ไหล,บาง ผ่า ยื่รึง หลอ,บง บอง จิง ก้ง รื่ง รอย,1.000000,1.200000,0.714286,0.809524,บาง ผ่า ยื่ง หลอ,บง บอง จิง ก้ง รื่ง รอย,1.000000,1.200000,0.761905,0.809524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,421,ยิ่ง ทำให้ ใจ ฉัน มัน ยิ่ง ทรมาน,ชห้ ฉัด ึงโช บ้า,ยั่ง คำ ให้ ใจ ห ฉัน ดัง โทรม รั,1.000000,1.000000,0.750000,0.437500,ให้ ฉัด โช บ้า,ยั่ง คำ ให้ ใจ หา ฉัน ดัง โทรม รับ,1.000000,1.000000,0.718750,0.468750
422,422,แต่ ก็ เหมือนว่า แค่ แกล้งทำ,แด กม เนิน ว่า แค่ กลาง ฉา,ต้ง ต่าง ก็ เหมือนว่า แค่ แก ลาง เท ำ,1.200000,1.200000,0.464286,0.392857,แด กม เนิน ว่า แค่ กลาง ฉา,ต้ง ต่าง ก็ เหมือนว่า แค่ แก ลาง เท ทำ,1.200000,1.200000,0.464286,0.428571
423,423,อากาศ ชื้น นะ คืนนี้ ระวัง หนาว นะ แบบนี้,อยาก ฉิง ลึก ถึ มี หู หวัง เดา นอกจากนี้,บอด กอด ฉัน เมื่อ คืง มี วงวัง เห รามา บอบ ห มือ,1.125000,1.500000,0.658537,0.804878,อยาก ฉิง ลึก ถึ มี หู หวัง เดา นอกจากนี้,บอด กอด ฉัน เมื่อ คือ มี วงวัง เห รามา บอบ หา มือ,1.125000,1.500000,0.658537,0.804878
424,424,ว่า ฉันนั้น เป็น อย่างไร,หวะ ฉัน นั่น เปน อย่างไร,ว่า ฉันนั้น เป็น อย่างไร,1.000000,0.000000,0.250000,0.000000,หวะ ฉัน นั่น เปน อย่างไร,ว่า ฉันนั้น เป็น อย่างไร,1.000000,0.000000,0.250000,0.000000


In [ ]:
pd.DataFrame(all_data).to_csv("/content/prediction1.csv",sep=",")